In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

Using backend: pytorch


In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 transform=T.ToSparseTensor())
dataset

PygNodePropPredDataset()

In [3]:
torch.cuda.is_available()

True

In [4]:
data = dataset[0]
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [5]:
split_idx = dataset.get_idx_split()
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [6]:
data.adj_t = data.adj_t.to_symmetric()
data.adj_t

SparseTensor(row=tensor([     0,      0,      0,  ..., 169341, 169342, 169342]),
             col=tensor([   411,    640,   1162,  ..., 163274,  27824, 158981]),
             size=(169343, 169343), nnz=2315598, density=0.01%)

In [7]:
row, col, _ = data.adj_t.coo()

In [8]:
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [9]:
edge_index.t()

tensor([[   411,      0],
        [   640,      0],
        [  1162,      0],
        ...,
        [163274, 169341],
        [ 27824, 169342],
        [158981, 169342]])

In [10]:
edge_index.shape

torch.Size([2, 2315598])

In [11]:
edge_index[:, 0]

tensor([411,   0])

In [12]:
edge_index.t()[0]

tensor([411,   0])

In [13]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self):
        super(LinkPredictor, self).__init__()

    def forward(self, x_i, x_j):
        x = torch.sum(torch.mul(x_i, x_j), dim=1)
        x = torch.clamp(x, max=6, min=-6)
        return torch.sigmoid(x)

In [14]:
device=0
log_steps=1
num_layers=2
hidden_channels=100
dropout=0.5
batch_size=128 * 1024
lr=0.005
epochs=100
eval_steps=2
runs=10

In [15]:
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

In [16]:
adj_t = data.adj_t.to(device)

In [17]:
torch.manual_seed(12345)

In [18]:
model = SAGE(hidden_channels, hidden_channels,
             hidden_channels, num_layers,
             dropout).to(device)

In [19]:
emb = torch.nn.Embedding(data.num_nodes, hidden_channels).to(device)

In [20]:
predictor = LinkPredictor().to(device)

In [21]:
emb.weight.size(0), emb.weight.size(1)

(169343, 100)

In [22]:
torch.nn.init.xavier_uniform_(emb.weight)

Parameter containing:
tensor([[ 0.0038, -0.0002,  0.0026,  ...,  0.0023,  0.0027,  0.0046],
        [ 0.0017, -0.0015,  0.0032,  ..., -0.0013,  0.0016,  0.0002],
        [-0.0028,  0.0036,  0.0030,  ..., -0.0023, -0.0009,  0.0002],
        ...,
        [-0.0008,  0.0051, -0.0049,  ...,  0.0044, -0.0042, -0.0022],
        [-0.0045, -0.0059,  0.0038,  ...,  0.0044,  0.0035, -0.0022],
        [-0.0031,  0.0025,  0.0058,  ...,  0.0010, -0.0040,  0.0003]],
       device='cuda:0', requires_grad=True)

In [23]:
model.reset_parameters()

In [24]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(predictor.parameters()), lr=lr)

In [25]:
def train(model, predictor, x, adj_t, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = edge_index.t().to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='sparse')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [26]:
for epoch in range(1, 1 + epochs):
    loss = train(model, predictor, emb.weight, adj_t,
                 optimizer, batch_size)
    
    if epoch % eval_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}')

Epoch: 02, Loss: 1.0239
Epoch: 04, Loss: 0.9434
Epoch: 06, Loss: 0.8932
Epoch: 08, Loss: 0.8603
Epoch: 10, Loss: 0.8368
Epoch: 12, Loss: 0.8214
Epoch: 14, Loss: 0.8104
Epoch: 16, Loss: 0.8034
Epoch: 18, Loss: 0.7977
Epoch: 20, Loss: 0.7939
Epoch: 22, Loss: 0.7891
Epoch: 24, Loss: 0.7858
Epoch: 26, Loss: 0.7836
Epoch: 28, Loss: 0.7811
Epoch: 30, Loss: 0.7795
Epoch: 32, Loss: 0.7779
Epoch: 34, Loss: 0.7762
Epoch: 36, Loss: 0.7741
Epoch: 38, Loss: 0.7736
Epoch: 40, Loss: 0.7718
Epoch: 42, Loss: 0.7445
Epoch: 44, Loss: 0.7244
Epoch: 46, Loss: 0.7380
Epoch: 48, Loss: 0.7226
Epoch: 50, Loss: 0.7021
Epoch: 52, Loss: 0.7089
Epoch: 54, Loss: 0.6940
Epoch: 56, Loss: 0.6921
Epoch: 58, Loss: 0.6808
Epoch: 60, Loss: 0.6725
Epoch: 62, Loss: 0.6783
Epoch: 64, Loss: 0.6571
Epoch: 66, Loss: 0.6453
Epoch: 68, Loss: 0.6322
Epoch: 70, Loss: 0.6285
Epoch: 72, Loss: 0.6216
Epoch: 74, Loss: 0.6158
Epoch: 76, Loss: 0.6067
Epoch: 78, Loss: 0.6012
Epoch: 80, Loss: 0.5986
Epoch: 82, Loss: 0.5960
Epoch: 84, Loss:

In [27]:
model.eval()

SAGE(
  (convs): ModuleList(
    (0): SAGEConv(100, 100)
    (1): SAGEConv(100, 100)
  )
)

In [28]:
emb.weight

Parameter containing:
tensor([[-0.2785, -0.2300,  0.4181,  ..., -0.1568,  0.2691,  0.2486],
        [ 0.0381,  0.0864, -0.1446,  ...,  0.0701, -0.0822, -0.1757],
        [ 0.0212,  0.0355,  0.1232,  ..., -0.0676, -0.0361,  0.0587],
        ...,
        [-0.0600,  0.0068,  0.2029,  ..., -0.1677,  0.0559,  0.0954],
        [ 0.0337,  0.0124,  0.0238,  ..., -0.1131,  0.1297,  0.0433],
        [ 0.1822,  0.2886, -0.4474,  ..., -0.0452,  0.4547, -0.2832]],
       device='cuda:0', requires_grad=True)

In [29]:
h = model(emb.weight, adj_t)

In [30]:
predictor(h[0:2], h[411:413])

tensor([0.9975, 0.0025], device='cuda:0', grad_fn=<SigmoidBackward>)

In [31]:
predictor(h[0:2], h[640:642])

tensor([0.9975, 0.9975], device='cuda:0', grad_fn=<SigmoidBackward>)

In [32]:
np.save('../../models/graphsage_link_pred/full_cos_graphsage_linkpred_13_h.npy', h.cpu().detach().numpy())

In [33]:
h_new = h.cpu().detach()

In [34]:
predictor(h_new[0:2], h_new[411:413])

tensor([0.9975, 0.0025])

In [35]:
torch.save(model, '../../models/graphsage_link_pred/13_model.pt')

In [36]:
torch.save(emb, '../../models/graphsage_link_pred/13_embedding.pt')